# Toronto Web Scraping Code

## Get the Source from WikiPedia page

## Imports

In [9]:
# standard computing libraries
import pandas as pd
import numpy as np
import json

# webscraping
from bs4 import BeautifulSoup
import requests

# geocoder for retrieving coordinates of postcodes
import geocoder

# k-means clustering
from sklearn.cluster import KMeans

# folium for maps
import folium

# geopandas for advanced geojson handling
import geopandas

# library to access overpass api in a more convenient way
import overpy

# ipython command to use matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'geocoder'

In [10]:
!pip install geocoder


     |████████████████████████████████| 102kB 12.2MB/s ta 0:00:01


In [11]:
# standard computing libraries
import pandas as pd
import numpy as np
import json

# webscraping
from bs4 import BeautifulSoup
import requests

# k-means clustering
from sklearn.cluster import KMeans

# folium for maps
import folium

# geopandas for advanced geojson handling
import geopandas

# library to access overpass api in a more convenient way
import overpy

# ipython command to use matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'folium'

In [12]:
! pip install folium

     |████████████████████████████████| 92kB 6.8MB/s eta 0:00:011


In [13]:
# geopandas for advanced geojson handling
import geopandas

# library to access overpass api in a more convenient way
import overpy

# ipython command to use matplotlib
%matplotlib inline

ModuleNotFoundError: No module named 'geopandas'

In [14]:
! pip install geopandas

     |████████████████████████████████| 931kB 15.3MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 27.3MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 43.4MB/s eta 0:00:01


In [15]:
postcode = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [16]:

postcodesoup = BeautifulSoup(postcode.content)

In [17]:
list_postcodes = list()

for tr in postcodesoup.find_all("tr")[2:-5]:
    
    row = (td.text for td in tr.find_all("td"))
    
    list_postcodes.append(row)

## use pandas to read the table into a pandas dataframe.

In [18]:

df_raw = pd.DataFrame(list_postcodes, columns=["Postcode","Borough","Neighbourhood"])

In [19]:
def concatNeighbourhood(row):
    
    return pd.Series(dict(Borough = row['Borough'].unique()[0],
                        Neighbourhood = ', '.join(row['Neighbourhood'])))

In [20]:
df_post = (df_raw
            .query('Borough != "Not assigned"')
            .assign(Neighbourhood = lambda x: x["Neighbourhood"].str[:-1].replace('Not assigned', x["Borough"]))
            .groupby('Postcode').apply(concatNeighbourhood)
            .reset_index()
)

df_post.head()

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [21]:
df_post.shape

(178, 3)

In [22]:
df_post.head(30)

,Postcode,Borough,Neighbourhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
5,M1J\n,Scarborough\n,Scarborough Village
6,M1K\n,Scarborough\n,Kennedy Park / Ionview / East Birchmount Park
7,M1L\n,Scarborough\n,Golden Mile / Clairlea / Oakridge
8,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village West
9,M1N\n,Scarborough\n,Birch Cliff / Cliffside West
